In [ ]:
## mi codigo
def solve_1D_BPP_WB(cargo, deferred_items = None, attempted_combinations = None , number_of_opened_uld = None, packed_ULDs = None, open_new_uld = False, open_new_extra_uld = False, deferred_items_optimal_flag = False):
    """
    Solve 1D Bin Packing Problem combined with Weight and Balance
    """
    m = Model("1D_BPP_WB")

    '''Parameters'''
    
    loadfactor = 0.8
    a_lat_TOW = 0.5
    b_lat_TOW = 0.5
    a_lat_LW = 0.5
    b_lat_LW = 0.5

    '''Open a new extra ULD compared to the actual set of ULDs [Feedback loop]'''

    if open_new_extra_uld is True:
        max_index = max([j.index for j in cargo.uld])
        extra_PMC = ULD(max_index + 1, 'PMC', 'PMC' + '-' + str(max_index + 1))
        extra_AKE = ULD(max_index + 2, 'AKE', 'AKE' + '-' + str(max_index + 2))
        cargo.uld.append(extra_PMC)
        cargo.uld.append(extra_AKE)
        cargo.define_parameters_ULD()

        # Decision Variables inlcuimos z sacamos w, abajo se define spbi
        f = {}
        u = {}
        p = {}
        z = {}

        for j in cargo.uld:
            for t in aircraft.loadlocations:
                f[j.index, t.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'f_{j.index}_{t.index}')

        for j in cargo.uld:
            u[j.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'u_{j.index}')

        for i in cargo.items:
            for j in cargo.uld:
                p[i.index, j.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'p_{i.index}_{j.index}')

        for i in cargo.items:
            for j in cargo.uld:
                for t in aircraft.loadlocations:
                    # Apply COL/CRT restrictions
                    if str(aircraft.aircraft_type) in ['772', '77W']:
                        if (i.COL == 1 or i.CRT == 1) and (t in aircraft.loadlocations_C1_C2 or t in aircraft.loadlocations_C3_C4):
                            continue
                    elif str(aircraft.aircraft_type) in ['789', '781']:
                        if (i.COL == 1 or i.CRT == 1) and t in aircraft.loadlocations_C3_C4:
                            continue
                    z[i.index, j.index, t.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'z_{i.index}_{j.index}_{t.index}')

        print('---------------------------------------------------------------------------')
        return results_1D_BPP_WB, ZFW_index, TOW_index, loadfactor_dict

In [ ]:
## para incluir el peso del uld vacio
# cambiar varias restricciones
# 1. Position Weight Limit Constraints (C7_1 and C7_2):
# 2. Compartment Weight Limit Constraints (C_Added_1, C_Added_2, C_Added_3, C_Added_4, C_Added_5, C_Added_6):
# 3. Maximum Payload Limit Constraint (C8):
# 4. Lateral Balance Constraints at Takeoff (C9_1, C9_2):
# 5. Lateral Balance Constraints at Landing (C10_1, C10_2):
# 6. Longitudinal TOW Constraints (CG Envelope) (C11_1, C11_2):
# 7. Longitudinal ZFW Constraints (CG Envelope) (C12_1, C12_2):
# 8. %MAC Objective Function (ZFW_index_obj):
# 9. Results Section (Weight Calculations):

## asi esta el codigo ahora
for t in aircraft.loadlocations:
    m.addConstr(quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld)
                <= aircraft.define_max_weight_postion(t), name=f'C7_1_{t.index}')
    
for t in aircraft.loadlocations:
    m.addConstr(quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT)
                <= aircraft.define_max_weight_postion(t), name=f'C7_2_{t.index}')

## este es el update 
for t in aircraft.loadlocations:
    m.addConstr(quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld)
                <= aircraft.define_max_weight_postion(t), name=f'C7_{t.index}')

In [ ]:
z = {}
for i in cargo.items:
    for j in cargo.uld:
        for t in aircraft.loadlocations:
            if str(aircraft.aircraft_type) in ['772', '77W']:
                if (i.COL == 1 or i.CRT == 1) and (t in aircraft.loadlocations_C1_C2 or t in aircraft.loadlocations_C3_C4):
                    continue
            elif str(aircraft.aircraft_type) in ['789', '781']:
                if (i.COL == 1 or i.CRT == 1) and t in aircraft.loadlocations_C3_C4:
                    continue
            z[(i.index, j.index, t.index)] = m.addVar(vtype=GRB.BINARY, name=f'z_{i.index}_{j.index}_{t.index}')
